In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from ema_workbench import load_results
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from ema_workbench import (Model, MultiprocessingEvaluator, SequentialEvaluator,
                           Constraint, Policy, Scenario)

from problem_formulation import get_model_for_problem_formulation

import time


In [2]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)



In [3]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [4]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter),
#lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)


CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [5]:
#enlisting policy levers, their types (RealParameter/IntegerParameter),
#lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [6]:
# #enlisting outcomes
# for outcome in dike_model.outcomes:
#     print(repr(outcome))
    
# specify outcomes
# note how we need to explicitely indicate the direction
dike_model.outcomes = [ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Total Investment Costs', kind=ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE)]

In [ ]:
#--- Search for candidate solutions through optimization
from ema_workbench.em_framework.optimization import (ArchiveLogger,
                                                     EpsilonProgress)
convergence_metrics  = [EpsilonProgress()]

with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.optimize(nfe=5000, searchover='levers',
                                 convergence=convergence_metrics,
                                 epsilons=[0.01,]*len(dike_model.outcomes))

[MainProcess/INFO] pool started with 8 workers
  0%|                                                 | 0/5000 [00:00<?, ?it/s]

In [ ]:
# def optimize(scenario, nfe, model, converge_metrics, epsilons):
#     #with SequentialEvaluator(model) as evaluator:
#     with MultiprocessingEvaluator(model) as evaluator:
#         results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
#                                      convergence=convergence_metrics,
#                                      epsilons=epsilons,
#                                      reference=scenario)

# Load data

In [ ]:
# # load the base case results
# experiments, outcomes = load_results('results/4000  base scenarios policy pf 1.tar.gz') 
# outcomes_df = pd.DataFrame.from_dict(outcomes)

# Assuming you have a CSV file called 'input.csv'
relevant_scenarios = pd.read_csv('results/relevant scenarios.csv')


In [ ]:
relevant_scenarios = relevant_scenarios.loc[:,:'discount rate 2']
scenarios =  [Scenario(f"{index}", **row) for index, row in relevant_scenarios.iterrows()]

for scenario in scenarios:
    print(scenario)
    
df_scenarios = pd.DataFrame(scenarios) 
df_scenarios.to_csv("data/scenarios_test.csv")     

In [ ]:
scenarios

In [ ]:


#specify uncertainties
dike_model.uncertainties = [IntegerParameter('A.0_ID flood wave shape', 0, 133),
                   RealParameter('A.1_Bmax', 30,350),
                   CategoricalParameter('A.1_Brate',(0.9,1.5,1000)),
                   RealParameter('A.1_pfail',0,1),
                   RealParameter('A.2_Bmax', 30,350),
                   CategoricalParameter('A.2_Brate',(0.9,1.5,1000)),
                   RealParameter('A.2_pfail',0,1),
                   RealParameter('A.3_Bmax', 30,350),
                   CategoricalParameter('A.3_Brate',(0.9,1.5,1000)),
                   RealParameter('A.3_pfail',0,1),
                   RealParameter('A.4_Bmax', 30,350),
                   CategoricalParameter('A.4_Brate',(0.9,1.5,1000)),
                   RealParameter('A.4_pfail',0,1),
                   RealParameter('A.5_Bmax', 30,350),
                   CategoricalParameter('A.5_Brate',(0.9,1.5,1000)),
                   RealParameter('A.5_pfail',0,1),
                   CategoricalParameter('discount rate 1',(1.5,2.5,3.5,4.5)),
                   CategoricalParameter('discount rate 2',(1.5,2.5,3.5,4.5))]

dike_model.levers = [IntegerParameter('A.1_DikeIncrease',0,10),
                     IntegerParameter('A.2_DikeIncrease',0,10),
                     IntegerParameter('A.3_DikeIncrease',0,10),
                     IntegerParameter('A.4_DikeIncrease',0,10),
                     IntegerParameter('A.5_DikeIncrease',0,2), #capped this variable to test if the model uses these specified levers
                     IntegerParameter('1_RfR 0',0,1),
                     IntegerParameter('2_RfR 0',0,1),
                     IntegerParameter('3_RfR 0',0,1),
                     IntegerParameter('4_RfR 0',0,1),
                     IntegerParameter('0_RfR 0',0,1),
                     IntegerParameter('EWS_DaysToThreat',0,4)]

# specify outcomes
# note how we need to explicitely indicate the direction
dike_model.outcomes = [ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Total Investment Costs', kind=ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE)]




In [ ]:
#--- Search for candidate solutions through optimization
from ema_workbench.em_framework.optimization import (ArchiveLogger,
                                                     EpsilonProgress)
convergence_metrics = [
    ArchiveLogger(
        "./archives",
        [l.name for l in dike_model.levers],
        [o.name for o in dike_model.outcomes],
        base_filename="test_1.tar.gz",
    ),
    EpsilonProgress(),
]

with MultiprocessingEvaluator(dike_model) as evaluator:
    results, convergence = evaluator.optimize(nfe=5000, searchover='levers',
                                 convergence=convergence_metrics,
                                 epsilons=[0.01,]*len(dike_model.outcomes))

In [ ]:
# This cell contains the code to run the optimization. Results were saved and therefore it is commented out.


ema_logging.log_to_stderr(ema_logging.INFO)

# The Hypervolume space has already been defined in the get_problem_formulation_altered function
convergence_metrics = [EpsilonProgress()]

nfe = 7500
results_deep = []
convergence_all = []

for scenario in scenarios:
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results_runs, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                        epsilons=[0.1,]*len(dike_model.outcomes))
                                        
        
        results_deep.append(results_runs)
        convergence_all.append(convergence)

# from ema_workbench import save_results

# for i in range(len(results_deep)):
#     save_results((results_deep[i], convergence_all[i]), f'../results/mordm_7500_rp_scenario{i}.tar.gz')

In [ ]:
# This cell contains the code to run the optimization. Results were saved and therefore it is commented out.


ema_logging.log_to_stderr(ema_logging.INFO)

# The Hypervolume space has already been defined in the get_problem_formulation_altered function
convergence_metrics = [HyperVolume.from_outcomes(dike_model.outcomes),
                       EpsilonProgress()]

nfe = 7500
results_deep = []
convergence_all = []

for scenario in scenarios:
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results_runs, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                        epsilons=[1e7, 1e6, 0.00001],
                                        convergence=convergence_metrics, reference=scenario)
                                        
        
        results_deep.append(results_runs)
        convergence_all.append(convergence)

from ema_workbench import save_results

for i in range(len(results_deep)):
    save_results((results_deep[i], convergence_all[i]), f'../results/mordm_test{i}.tar.gz')

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.optimization import (
    ArchiveLogger,
    EpsilonProgress,
    to_problem,
    epsilon_nondominated,
)

results = []
nfes = 100

for scenario in scenarios:
    convergence_metrics = [
                            ArchiveLogger(
                                "./results",
                                [l.name for l in dike_model.levers],
                                [o.name for o in dike_model.outcomes],
                                base_filename=f"test_test_{scenario.name}_seed.tar.gz",
                            ),
                            EpsilonProgress(),
                        ]
    epsilons = [0.1,]*len(dike_model.outcomes)
    
    results.append(optimize(scenario, nfes, dike_model, convergence_metrics, epsilons))

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.optimization import (
    ArchiveLogger,
    EpsilonProgress,
    to_problem,
    epsilon_nondominated,
)

ema_logging.log_to_stderr(ema_logging.INFO)

def optimize(scenario, nfe, model, epsilons):
    results = []
    convergences = []
    problem = to_problem(model, searchover="levers")

    with MultiprocessingEvaluator(model) as evaluator:
        convergence_metrics = [
            ArchiveLogger(
                "./results",
                [l.name for l in model.levers],
                [o.name for o in model.outcomes],
                base_filename=f"test_test_{scenario.name}_seed.tar.gz",
            ),
            EpsilonProgress(),
        ]

        result, convergence = evaluator.optimize(
            nfe=nfe,
            searchover='levers',
            convergence=convergence_metrics,
            epsilons=epsilons,
            reference=scenario
        )

        results.append(result)
        convergences.append(convergence)

    # Merge the results using a non-dominated sort
    reference_set = epsilon_nondominated(results, epsilons, problem)

    return reference_set, convergences

results = []
for scenario in scenarios:
    epsilons = [0.05] * len(dike_model.outcomes)

    # Note that 100000 nfe is rather low to ensure proper convergence
    results.append(optimize(scenario, 100, dike_model, epsilons))

print(results)


In [ ]:
# from ema_workbench import MultiprocessingEvaluator, ema_logging
# from ema_workbench.em_framework.evaluators import BaseEvaluator

# from ema_workbench.em_framework.optimization import (ArchiveLogger,
#                                                      EpsilonProgress,
#                                                      to_problem, epsilon_nondominated)

# ema_logging.log_to_stderr(ema_logging.INFO)

# def optimize(scenario, nfe, model, epsilons):
#     results = []
#     convergences = []
#     problem = to_problem(model, searchover="levers")

#     with MultiprocessingEvaluator(model) as evaluator:
#         for i in range(5):
#             print('for loop begint')
#             convergence_metrics = [
#                 ArchiveLogger(
#                     "./results",
#                     [l.name for l in model.levers],
#                     [o.name for o in model.outcomes],
#                     base_filename=f"test_test_{scenario.name}_seed_{i}.tar.gz",
#                 ),
#                 EpsilonProgress(),
#             ]

#             result, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
#                                          convergence=convergence_metrics,
#                                          epsilons=epsilons,
#                                          reference=scenario)
#             print('evaluator.optimize is gerund')
#             results.append(result)
#             convergences.append(convergence)
    
#     # merge the results using a non-dominated sort  
#     reference_set = epsilon_nondominated(results, epsilons, problem)    
            
#     return reference_set, convergences


# results = []
# for scenario in scenarios:
#     epsilons = [0.05,]*len(dike_model.outcomes)
    
#     # note that 100000 nfe is again rather low to ensure proper convergence
#     results.append(optimize(scenario, 100, dike_model, epsilons))
#     print(results)